In [19]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import logit, loglogit, piecewiseFormula, nested
from biogeme.models import lognested
from biogeme.results import bioResults, pickle

In [14]:
df = pd.read_table("new_lpmc02.dat", index_col=0)
database = db.Database("LPMC",df)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

In [17]:
# Model

  
# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public = DefineVariable('cost_public', cost_transit ,database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 * car_ownership +\
                    Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1, 2: 1, 3: 1, 4: 1}


#Definitions of nests
N_SM = N_SM, [1, 2]
N_MOTOR = N_MOTOR, [3, 4]

nests = N_SM, N_MOTOR

In [18]:
sum_weights = database.data['Weights'].sum()
S = database.getSampleSize()
sample_normalized_weight = Weights * S / sum_weights

In [35]:
# market share prediction 

prob_walking = nested(V, av, nests, 1)
prob_cycling = nested(V, av, nests, 2)
prob_public = nested(V, av, nests, 3)
prob_car = nested(V, av, nests, 4)


simulate = {'Prob. walking': prob_walking,
            'Prob. cycling': prob_cycling,
            'Prob. public': prob_public,
            'Prob. car': prob_car,
            'Weighted prob. walking': sample_normalized_weight * prob_walking,
            'Weighted prob. cycling': sample_normalized_weight * prob_cycling,
            'Weighted prob. public': sample_normalized_weight * prob_public,
            'Weighted prob. car': sample_normalized_weight * prob_car
           }

output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_nested_lpmc_sm_motor")
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)
    
# # delete previously saved html and pickle
# for file in glob.glob(f"{filepath}*"):
#     os.remove(file)

biogeme  = bio.BIOGEME(database, simulate)
#biogeme.modelName = filepath

betas = biogeme.freeBetaNames
results = bioResults(pickleFile=f"{filepath}.pickle")

beta_values = results.getBetaValues()

In [36]:
beta_values

{'CarOwn_2': -0.19899460851371756,
 'CarOwn_3': -0.2397633261478376,
 'CarOwn_4': 0.818724028999332,
 'Constant2': -2.3093897372032033,
 'Constant3': 2.004536722048567,
 'Constant4': 0.037798345238455704,
 'Cost': -0.09169334451163441,
 'LAMBDA': -0.00932632088657424,
 'N_MOTOR': 1.6407523088267113,
 'N_SM': 1.0,
 'Time_Age_1': -0.05091888514234194,
 'Time_Age_2': -0.028169361349548845,
 'Time_Age_3': -0.03422841171350653,
 'Time_Age_4': -0.04272785311027614,
 'Total_TT1': -2.7818384822026103,
 'Total_TT2': -1.2945230913044004,
 'Total_TT3': -0.6289411599628236,
 'Total_TT4': -0.979412044253497}

In [37]:
simulated_values = biogeme.simulate(beta_values)

marketShare_walking = 100 * simulated_values['Weighted prob. walking'].mean()
marketShare_cycling = 100 * simulated_values['Weighted prob. cycling'].mean()
marketShare_public = 100 * simulated_values['Weighted prob. public'].mean()
marketShare_car = 100 * simulated_values['Weighted prob. car'].mean()

In [39]:
# conf interval
b = results.getBetasForSensitivityAnalysis(betas, size=100)
left, right = biogeme.confidenceIntervals(b, 0.9)

In [41]:
pd.DataFrame(left)

,Prob. walking,Prob. cycling,Prob. public,Prob. car,Weighted prob. walking,Weighted prob. cycling,Weighted prob. public,Weighted prob. car
0,0.164,0.0493,0.467,0.15,0.175,0.0528,0.5,0.161
1,0.000103,0.0105,0.907,0.0236,0.00011,0.0112,0.971,0.0252
2,0.0268,0.0116,0.0279,0.843,0.0287,0.0125,0.0299,0.902
3,0.04,0.00541,0.066,0.773,0.0428,0.00579,0.0707,0.828
4,0.555,0.0142,0.0891,0.201,0.594,0.0152,0.0953,0.215
...,...,...,...,...,...,...,...,...
4995,0.177,0.0589,0.193,0.426,0.152,0.0506,0.166,0.367
4996,0.033,0.0201,0.0231,0.802,0.0284,0.0173,0.0198,0.689
4997,0.567,0.0384,0.0989,0.0972,0.488,0.033,0.085,0.0836
4998,3.02e-05,0.0117,0.133,0.758,2.59e-05,0.01,0.114,0.652


In [42]:
pd.DataFrame(right)

,Prob. walking,Prob. cycling,Prob. public,Prob. car,Weighted prob. walking,Weighted prob. cycling,Weighted prob. public,Weighted prob. car
0,0.229,0.0973,0.582,0.248,0.245,0.104,0.623,0.265
1,0.00141,0.0262,0.962,0.0516,0.00151,0.028,1.03,0.0552
2,0.05,0.0341,0.0614,0.927,0.0535,0.0365,0.0657,0.992
3,0.079,0.0191,0.14,0.873,0.0845,0.0205,0.15,0.934
4,0.654,0.0564,0.14,0.292,0.7,0.0604,0.15,0.312
...,...,...,...,...,...,...,...,...
4995,0.256,0.149,0.266,0.515,0.22,0.128,0.229,0.442
4996,0.0611,0.0714,0.0525,0.912,0.0525,0.0614,0.0452,0.784
4997,0.675,0.147,0.152,0.224,0.58,0.127,0.131,0.192
4998,0.000685,0.0266,0.205,0.845,0.000589,0.0228,0.176,0.727


In [60]:
df.head()["travel_mode"], df.columns # sanity check

(0    3
 1    3
 2    4
 3    4
 4    1
 Name: travel_mode, dtype: int64,
 Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
        'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
        'travel_year', 'travel_month', 'travel_date', 'day_of_week',
        'start_time', 'age', 'female', 'driving_license', 'car_ownership',
        'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
        'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
        'dur_driving', 'cost_transit', 'cost_driving_fuel',
        'cost_driving_ccharge', 'driving_traffic_percent', 'Weights',
        'cost_public', 'dur_public', 'cost_driving'],
       dtype='object'))

In [57]:
lst_marketShares = [marketShare_walking, marketShare_cycling, marketShare_public, marketShare_car]
temp_names = ["marketShare_walking", "marketShare_cycling", "marketShare_public", "marketShare_car"]

print("Predicted market shares:\n")
for i in range(len(temp_names)):
    l = left[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    r = right[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    print(f"{temp_names[i]}: {lst_marketShares[i]:.2f}% ({l:.2f}%, {r:.2f}%)")

Predicted market shares:

marketShare_walking: 17.59% (15.55%, 19.95%)
marketShare_cycling: 2.84% (2.20%, 5.95%)
marketShare_public: 36.43% (33.25%, 40.44%)
marketShare_car: 43.14% (35.90%, 45.69%)


In [61]:
# compare with actual choices
# ... 